In [1]:
import pandas as pd

In [4]:
dbDf = pd.read_csv('static/data/database.csv')

dbDf.head()

,index,time_observed_at,species_guess,place_guess,location,iconic_taxon_name,name,preferred_common_name,wikipedia_url,category
0,0,2021-10-03T19:02:00-04:00,beefsteak plant,"Stone Throw Ct, Stone Mountain, GA, US","{33.775305,-84.1047971667}",Plantae,Perilla frutescens,beefsteak plant,http://en.wikipedia.org/wiki/Perilla_frutescens,Introduced
1,1,2021-10-03T22:23:24-04:00,Mediterranean House Gecko,"Rocking Horse Ln, Pooler, GA, US","{32.165454221,-81.2527658791}",Reptilia,Hemidactylus turcicus,Mediterranean House Gecko,http://en.wikipedia.org/wiki/Mediterranean_hou...,Introduced
2,2,2021-10-03T14:10:58-04:00,common fig,"Elm Dr, Forest Park, GA, US","{33.5914216667,-84.36065}",Plantae,Ficus carica,common fig,http://en.wikipedia.org/wiki/Common_fig,Introduced
3,3,2021-10-02T13:28:41-04:00,Brown Anole,"Montgomery St, Savannah, GA, US","{32.0475278333,-81.1092}",Reptilia,Anolis sagrei,Brown Anole,http://en.wikipedia.org/wiki/Brown_anole,Introduced
4,4,2021-10-03T18:02:00-04:00,Heavenly bamboo,"Morrow, GA 30260, USA","{33.5784833333,-84.3368833333}",Plantae,Nandina domestica,Heavenly bamboo,https://en.wikipedia.org/wiki/Nandina,Introduced


In [18]:
test_df = dbDf.groupby(by='preferred_common_name', as_index=False).sum()
#test_df = test_df['preferred_common_name','sum']
test_df.reset_index
test_df.columns

Index(['preferred_common_name', 'index'], dtype='object')

In [36]:
import json

final_children = []
for category in dbDf['category'].unique():
    start_children = []
    category_df = dbDf[(dbDf['category'] == category)]
    for name in dbDf['iconic_taxon_name'].unique():
        taxon_children = []
        taxon_df = category_df[(category_df['iconic_taxon_name'] == name)]
        taxon_df = taxon_df.groupby(by='preferred_common_name', as_index=False).sum()
        for index, taxon in taxon_df.iterrows():
            temp_dict = {"name": taxon['preferred_common_name'], "value": taxon['index']}
            taxon_children.append(temp_dict)
        taxon_dict = {"name": name, "children": taxon_children}
        start_children.append(taxon_dict)
    start_json = {"name": category, "children": start_children}
    final_children.append(start_json)
    with open(f'static/data/data_{category}.json', 'w+') as outfile:
        json.dump(start_json, outfile)